In [1]:
# DuckDB quick query helper for the Store Sales parquet

from pathlib import Path
import duckdb
import pandas as pd

# Adjust if your parquet is elsewhere
PARQUET_PATH = Path("../data/Store_Sales_Price_Elasticity_Promotions_Data.parquet").resolve()
TABLE_NAME = "sales"

if not PARQUET_PATH.exists():
    raise FileNotFoundError(f"Parquet not found: {PARQUET_PATH}")

# In-memory DuckDB connection (fast for ad-hoc queries)
con = duckdb.connect(database=":memory:")

# Create a view over the parquet (no need to import the whole file)
con.execute(f"CREATE OR REPLACE VIEW {TABLE_NAME} AS SELECT * FROM read_parquet('{PARQUET_PATH.as_posix()}')")

print("DuckDB ready")
print("Table:", TABLE_NAME)
print("Parquet:", PARQUET_PATH)


def run_sql(sql: str, *, limit: int | None = None) -> pd.DataFrame:
    """Run SQL against DuckDB and return a pandas DataFrame."""
    q = sql.strip().rstrip(";")
    if limit is not None:
        q = f"SELECT * FROM ({q}) t LIMIT {int(limit)}"
    return con.execute(q).df()


# Example (edit/remove)
run_sql(f"SELECT * FROM {TABLE_NAME} LIMIT 5")


DuckDB ready
Table: sales
Parquet: C:\Users\Recup\Documents\Polimi\PACS\DataAgent\data\Store_Sales_Price_Elasticity_Promotions_Data.parquet


,Store_Number,SKU_Coded,Product_Class_Code,Sold_Date,Qty_Sold,Total_Sale_Value,On_Promo
0,1320,6172800,22875,2021-11-02,3,56.849998,0
1,2310,6172800,22875,2021-11-03,1,18.950001,0
2,3080,6172800,22875,2021-11-03,1,18.950001,0
3,2310,6172800,22875,2021-11-06,1,18.950001,0
4,4840,6172800,22875,2021-11-07,1,18.950001,0


In [5]:
"What is the total sales in october 2022 ?"

run_sql("""
SELECT
  SUM(Total_Sale_Value) as Total_Sale
FROM sales
WHERE Sold_Date >= DATE '2022-10-01'
  AND Sold_Date <  DATE '2022-11-01'
""")

,Total_Sale
0,462807.737202


In [7]:
"How many units we sold in september 2022 ?"

run_sql("""
SELECT
  SUM(Qty_Sold) as Total_Units
FROM sales
WHERE Sold_Date >= DATE '2022-09-01'
  AND Sold_Date <  DATE '2022-10-01'
""")


,Total_Units
0,32153.0


In [16]:
"Find product categories that decreased sales during the 2021 holiday season (November to December)"

run_sql("""
SELECT Product_Class_Code, SUM(Total_Sale_Value) as Total_Sale 
FROM sales 
WHERE Sold_Date >= DATE '2021-11-01' AND Sold_Date < DATE '2021-12-01' 
GROUP BY Product_Class_Code 
ORDER BY Total_Sale DESC
""")

,Product_Class_Code,Total_Sale
0,22975,95095.530096
1,22800,75044.439131
2,24425,61189.669664
3,24400,61180.949032
4,22900,53564.871561
5,22850,49455.339703
6,24375,41718.810003
7,22875,25103.339628
8,22925,17076.859786
9,22825,11950.189796


In [20]:
"How much was the variation in sales beween January to February in 2022"

run_sql("""
SELECT
  (SELECT SUM(Total_Sale_Value) FROM sales WHERE Sold_Date >= DATE '2022-01-01' AND Sold_Date < DATE '2022-02-01')
  -
  (SELECT SUM(Total_Sale_Value) FROM sales WHERE Sold_Date >= DATE '2022-02-01' AND Sold_Date < DATE '2022-03-01')
  AS Sales_Variation
""")

,Sales_Variation
0,54671.109787


In [22]:
"Which store has the most sales in october 2022 ?"

run_sql("""
SELECT Store_Number, SUM(Total_Sale_Value) as Total_Sale 
FROM sales 
WHERE Sold_Date >= DATE '2022-10-01' AND Sold_Date < DATE '2022-11-01' 
GROUP BY Store_Number 
ORDER BY Total_Sale DESC LIMIT 1
""")

,Store_Number,Total_Sale
0,2970,27922.329905


In [23]:
"List top 10 stores by average transaction value during December 2021"

run_sql("""
SELECT Store_Number, AVG(Total_Sale_Value) as Average_Transaction_Value 
FROM sales 
WHERE Sold_Date >= DATE '2021-12-01' AND Sold_Date < DATE '2022-01-01' 
GROUP BY Store_Number 
ORDER BY Average_Transaction_Value DESC LIMIT 10
""")

,Store_Number,Average_Transaction_Value
0,4400,42.095486
1,2970,29.879128
2,1210,24.459097
3,2750,23.893291
4,2200,23.773117
5,1650,23.473205
6,1540,23.364011
7,880,23.354056
8,990,23.293025
9,3300,23.179454


In [24]:
"Show me in a bar plot the monthly sales in 2022"

run_sql("""
SELECT DATE_TRUNC('month', Sold_Date) as month_start, SUM(Total_Sale_Value) as total_revenue 
FROM sales 
WHERE EXTRACT(YEAR FROM Sold_Date) = 2022 
GROUP BY DATE_TRUNC('month', Sold_Date) 
ORDER BY month_start ASC
""")

,month_start,total_revenue
0,2022-01-01,460090.187698
1,2022-02-01,405419.077911
2,2022-03-01,412956.587351
3,2022-04-01,371117.707672
4,2022-05-01,324769.278198
5,2022-06-01,324996.188044
6,2022-07-01,346132.107986
7,2022-08-01,368018.057711
8,2022-09-01,403554.177320
9,2022-10-01,462807.737202


In [25]:
"show store performance for Q1 2022 (January to March) in total revenue, ordered by total revenue."

run_sql("""
SELECT Store_Number, SUM(Total_Sale_Value) as Total_Revenue 
FROM sales 
WHERE Sold_Date >= DATE '2022-01-01' AND Sold_Date < DATE '2022-04-01' 
GROUP BY Store_Number 
ORDER BY Total_Revenue DESC
""")

,Store_Number,Total_Revenue
0,2970,93924.699715
1,1650,59678.759716
2,3300,58892.809644
3,3080,54900.039758
4,1210,54417.559776
5,1320,52148.949675
6,1540,47457.179678
7,2750,44678.069695
8,880,43990.549836
9,1100,41883.569766


In [32]:
"Give me by SKU level the top 3 products in January 2022"

run_sql("""
SELECT SKU_Coded, Product_Class_Code, SUM(Total_Sale_Value) as Total_Revenue
FROM sales
WHERE Sold_Date >= DATE '2022-01-01' AND Sold_Date < DATE '2022-02-01'
GROUP BY Product_Class_Code, SKU_Coded
ORDER BY Total_Revenue DESC
LIMIT 3
""")

,SKU_Coded,Product_Class_Code,Total_Revenue
0,6184900,24400,12054.159729
1,6194250,22975,11803.569909
2,6191400,22900,8597.490253


In [35]:
"Which stores sold more than 1000 items in December 2021? Show total and promo sales, then plot them with a visualization"

# In this schema, there is no 'Promo_Sale_Value' column.
# To get Promo sales, sum 'Total_Sale_Value' only where 'On_Promo'==1.
# To get total sales, sum 'Total_Sale_Value' for all rows.

run_sql("""
SELECT 
    Store_Number, 
    SUM(Total_Sale_Value) AS Total_Revenue,
    SUM(CASE WHEN On_Promo = 1 THEN Total_Sale_Value ELSE 0 END) AS Promo_Revenue
FROM sales
WHERE Sold_Date >= DATE '2021-12-01' AND Sold_Date < DATE '2022-01-01'
GROUP BY Store_Number
HAVING SUM(Qty_Sold) > 1000
""")

,Store_Number,Total_Revenue,Promo_Revenue
0,1320,32853.569915,1762.250000
1,1540,37569.329945,2212.820011
2,4730,18747.209966,329.429997
3,1650,38965.520022,2194.920038
4,550,16161.550002,962.350015
5,4180,21071.019946,425.500008
6,1980,17433.159965,1033.020023
7,2750,35648.789999,1425.600025
8,990,24713.900014,1830.849999
9,2200,34780.070020,1920.350025


In [43]:
"Show me all the sales registered from January 2022 to January 2023, grouped by month. Then show me some insights about the sales and plot the data as bar plot - visualization"

run_sql("""
SELECT
  DATE_TRUNC('month', CAST(Sold_Date AS DATE)) AS month_start,
  SUM(Total_Sale_Value)                        AS total_revenue
FROM sales
WHERE CAST(Sold_Date AS DATE) >= DATE '2022-01-01'
  AND CAST(Sold_Date AS DATE) <  DATE '2023-01-01'
GROUP BY 1
ORDER BY 1 ASC
""")

,month_start,total_revenue
0,2022-01-01,460090.187698
1,2022-02-01,405419.077911
2,2022-03-01,412956.587351
3,2022-04-01,371117.707672
4,2022-05-01,324769.278198
5,2022-06-01,324996.188044
6,2022-07-01,346132.107986
7,2022-08-01,368018.057711
8,2022-09-01,403554.177320
9,2022-10-01,462807.737202
